In [1]:
from pathlib import Path


root_path = Path("/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/")

fpath_output = Path("/home/jovyan/InputData/")
fpath_output.mkdir(exist_ok=True, parents=True)




In [2]:
def get_subdirs(path: Path):
    list_paths = []
    for path in Path(path).iterdir():
        if path.is_dir():
            list_paths.append(path)
    
    return list_paths


dirs_acdc = get_subdirs(root_path)
print(dirs_acdc)


[PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/DCM'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/HCM'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/MINF'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/NOR')]


In [3]:

import nibabel as nib

import numpy as np

def crop_image(imgarr, cropped_size):
    
    imgdims = np.array(imgarr.shape)//2
    xstart = imgdims[0] - cropped_size//2
    xend = imgdims[0] + cropped_size//2
    
    ystart = imgdims[1] - cropped_size//2
    yend = imgdims[1] + cropped_size//2
    

    return imgarr[xstart:xend, ystart:yend,...]

def get_cropped_image(path: Path):
    
    img = nib.load(str(path / "image.nii.gz"))

    cropsize=128
    cropped_arr = crop_image(img.get_fdata(), cropsize)

    return np.squeeze(cropped_arr)

min_num_phases = 24
time_usf = 2

    

In [4]:

i=0
for curr_dir in dirs_acdc:
    print(curr_dir)
    list_patients = get_subdirs(curr_dir)
    
    for pat in list_patients:
        img = get_cropped_image(pat)
        
        if img.shape[3] < min_num_phases:
            continue
        
        img = img[...,1:min_num_phases:time_usf]
        
        for islice in range(img.shape[2]):
            dat = np.squeeze(img[:,:,islice,:])
            nii = nib.Nifti1Image(dat, np.eye(4))
            nib.save(nii, str(fpath_output / "img_{}.nii".format(i)))
            i += 1
        

/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/DCM


KeyboardInterrupt: 

In [4]:
import sirf.Gadgetron as pMR
import numpy as np


fname_template = fpath_output / "template/cine_128_30ph_bc.h5"
template_data = pMR.AcquisitionData(str(fname_template))

template_data = pMR.preprocess_acquisition_data(template_data)
template_phase = template_data.get_ISMRMRD_info("phase")

index_all_acquis = np.arange(template_data.number())

index_interesting_acquis = template_phase < (min_num_phases// time_usf)
index_interesting_acquis = index_all_acquis[index_interesting_acquis]
template_data = template_data.get_subset(index_interesting_acquis)



Started reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..100%..
Finished reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5


In [5]:
import mathplus_p1_auxiliary_functions as aux 

def undersample_cartesian_data(ad: pMR.AcquisitionData):

    ky_index = ad.get_ISMRMRD_info('kspace_encode_step_1')
    cph_index = ad.get_ISMRMRD_info('phase')
    ky_num = int(np.max(ky_index)+1)
    cph_num = int(np.max(cph_index)+1)
    print(f'Nky {ky_num} - Ncph {cph_num}')

    R = 4
    F = int(ky_num/10)
    msk = aux.cartesian_mask([cph_num, ky_num, 1], R, sample_n=F)


    acq_us = ad.new_acquisition_data(empty=True)

    # Create raw data
    for cnd in range(cph_num):
        for ynd in range(ky_num):
            if msk[cnd, ynd, 0] == 1:
                cidx = np.where((ky_index == ynd) & (cph_index == cnd))[0]
                if len(cidx) > 0:
                    cacq = ad.acquisition(cidx)
                    acq_us.append_acquisition(cacq)
                else:
                    print(f'ky {ynd} - cph {cnd} not found')

    acq_us.sort()     

    return acq_us

print(template_data.number())
template_data = undersample_cartesian_data(template_data)
print(template_data.number())

SyntaxError: invalid syntax (mathplus_p1_auxiliary_functions.py, line 12)

In [10]:
import sirf.Reg as pReg


csm = pMR.CoilSensitivityData()
csm.calculate(template_data)
csm.fill(1 + 0. * 1j)

template_img = pMR.ImageData()
template_img.from_acquisition_data(template_data)

E = pMR.AcquisitionModel(acqs=template_data, imgs=template_img)
E.set_coil_sensitivity_maps(csm)

path_input = fpath_output

list_stacks = sorted(path_input.glob("img*"))

template_fwd = pMR.ImageData()
template_fwd.from_acquisition_data(template_data)

iy = 0
for f in list_stacks:
    print("GO {}/{}!".format(iy, len(list_stacks)))
    nii = pReg.ImageData(str(f))
    template_fwd.fill(nii.as_array())
    
    rd = E.forward(template_fwd)
    
    fout = fpath_output / "y_{}.h5".format(iy)
    rd.write(str(fout))
    iy += 1

GO 0/488!
GO 1/488!
GO 2/488!
GO 3/488!
GO 4/488!
GO 5/488!
GO 6/488!
GO 7/488!
GO 8/488!
GO 9/488!
GO 10/488!
GO 11/488!
GO 12/488!
GO 13/488!
GO 14/488!
GO 15/488!
GO 16/488!
GO 17/488!
GO 18/488!


KeyboardInterrupt: 